<a href="https://colab.research.google.com/github/LekshmiRM/python_assignments/blob/main/Data_acq_casestudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import numpy as np
import sqlite3

In [68]:
conn=sqlite3.connect("launches.db")
cursor=conn.cursor()

In [2]:
import requests

In [7]:
url='https://api.spacexdata.com/v4/launches'
response = requests.get(url)
data = response.json()


In [43]:
launches_df=pd.DataFrame(data)
launches_df=launches_df[["name", "date_utc", "success", "details", "rocket"]]
launches_df

launches_df

,name,date_utc,success,details,rocket
0,FalconSat,2006-03-24T22:30:00.000Z,False,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb
1,DemoSat,2007-03-21T01:10:00.000Z,False,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb
2,Trailblazer,2008-08-03T03:34:00.000Z,False,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb
3,RatSat,2008-09-28T23:15:00.000Z,True,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb
4,RazakSat,2009-07-13T03:35:00.000Z,True,None,5e9d0d95eda69955f709d1eb
...,...,...,...,...,...
200,Transporter-6,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69973a809d1ec
201,TTL-1,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69973a809d1ec
202,WorldView Legion 1 & 2,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69973a809d1ec
203,Viasat-3 & Arcturus,2022-12-01T00:00:00.000Z,None,None,5e9d0d95eda69974db09d1ed


Extract relevant columns: name, date_utc, success, details, rocket

**Convert date_utc to datetime and extract the year**

In [53]:
launches_df['date_utc']=pd.to_datetime(launches_df['date_utc'])
launches_df['year']=launches_df['date_utc'].dt.year
launches_df

,name,date_utc,success,details,rocket,year
0,FalconSat,2006-03-24 22:30:00+00:00,False,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006
1,DemoSat,2007-03-21 01:10:00+00:00,False,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007
2,Trailblazer,2008-08-03 03:34:00+00:00,False,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008
3,RatSat,2008-09-28 23:15:00+00:00,True,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008
4,RazakSat,2009-07-13 03:35:00+00:00,True,None,5e9d0d95eda69955f709d1eb,2009
...,...,...,...,...,...,...
200,Transporter-6,2022-12-01 00:00:00+00:00,None,None,5e9d0d95eda69973a809d1ec,2022
201,TTL-1,2022-12-01 00:00:00+00:00,None,None,5e9d0d95eda69973a809d1ec,2022
202,WorldView Legion 1 & 2,2022-12-01 00:00:00+00:00,None,None,5e9d0d95eda69973a809d1ec,2022
203,Viasat-3 & Arcturus,2022-12-01 00:00:00+00:00,None,None,5e9d0d95eda69974db09d1ed,2022


**Step 2: Load Rocket Metadata**

In [47]:
url1="https://api.spacexdata.com/v4/rockets"

In [48]:
response=requests.get(url1)
data=response.json()

**Extract id, name, type, active, and stages**

In [50]:
rocket_df=pd.DataFrame(data)
rocket_df.head()

rocket_df=rocket_df[["id","name","type","active","stages"]]
rocket_df


,id,name,type,active,stages
0,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
1,5e9d0d95eda69973a809d1ec,Falcon 9,rocket,True,2
2,5e9d0d95eda69974db09d1ed,Falcon Heavy,rocket,True,2
3,5e9d0d96eda699382d09d1ee,Starship,rocket,False,2


**Step 3: Merge Launch and Rocket Data**

In [54]:
merged_df=launches_df.merge(
    rocket_df,
    left_on="rocket",
    right_on="id",
    how="left"
)
merged_df.head()

,name_x,date_utc,success,details,rocket,year,id,name_y,type,active,stages
0,FalconSat,2006-03-24 22:30:00+00:00,False,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
1,DemoSat,2007-03-21 01:10:00+00:00,False,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
2,Trailblazer,2008-08-03 03:34:00+00:00,False,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
3,RatSat,2008-09-28 23:15:00+00:00,True,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2
4,RazakSat,2009-07-13 03:35:00+00:00,True,None,5e9d0d95eda69955f709d1eb,2009,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2


Add Simulated Country Information
● Add a new column country and randomly assign one of these values: ['USA', 'Russia', 'India', 'China', 'France']

In [58]:
countries=['USA','Russia','India','China','France']

merged_df["country"]=np.random.choice(countries,size=len(merged_df))
merged_df.head()

,name_x,date_utc,success,details,rocket,year,id,name_y,type,active,stages,country
0,FalconSat,2006-03-24 22:30:00+00:00,False,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,Russia
1,DemoSat,2007-03-21 01:10:00+00:00,False,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,Russia
2,Trailblazer,2008-08-03 03:34:00+00:00,False,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,China
3,RatSat,2008-09-28 23:15:00+00:00,True,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,France
4,RazakSat,2009-07-13 03:35:00+00:00,True,None,5e9d0d95eda69955f709d1eb,2009,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,False,2,USA


Step 5: Store Merged Data in SQLite3
● Use sqlite3 to create a connection and save the merged DataFrame as a table named launches
● Table should contain all merged columns including country **bold text**

In [70]:
merged_df.to_sql('launches',conn,index="false")
df_to_db=pd.read_sql('select * from launches LIMIT 5',conn)
df_to_db

,false,name_x,date_utc,success,details,rocket,year,id,name_y,type,active,stages,country
0,0,FalconSat,2006-03-24 22:30:00+00:00,0,Engine failure at 33 seconds and loss of vehicle,5e9d0d95eda69955f709d1eb,2006,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,0,2,Russia
1,1,DemoSat,2007-03-21 01:10:00+00:00,0,Successful first stage burn and transition to ...,5e9d0d95eda69955f709d1eb,2007,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,0,2,Russia
2,2,Trailblazer,2008-08-03 03:34:00+00:00,0,Residual stage 1 thrust led to collision betwe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,0,2,China
3,3,RatSat,2008-09-28 23:15:00+00:00,1,Ratsat was carried to orbit on the first succe...,5e9d0d95eda69955f709d1eb,2008,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,0,2,France
4,4,RazakSat,2009-07-13 03:35:00+00:00,1,None,5e9d0d95eda69955f709d1eb,2009,5e9d0d95eda69955f709d1eb,Falcon 1,rocket,0,2,USA


Step 6: Run SQL Queries on the Data to analyze
1. Launches by Country 2. Which year had the highest number of launches? 3. Top 5 Missions by Launch Count **bold text** **bold text** **bold text**

In [79]:
query="""select country,COUNT(*) AS t_launches FROM launches GROUP BY country ORDER BY t_launches DESC;"""

In [80]:
result = pd.read_sql_query(query, conn)

In [81]:
result

,country,t_launches
0,USA,46
1,China,46
2,France,40
3,India,38
4,Russia,35


In [82]:
q2 = """
SELECT year, COUNT(*) AS total_launches
FROM launches
WHERE year IS NOT NULL
GROUP BY year
ORDER BY total_launches DESC
LIMIT 1;
"""

In [84]:
result = pd.read_sql_query(q2, conn)

In [85]:
result

,year,total_launches
0,2022,62


In [87]:
q3 = """
SELECT name_x, COUNT(*) AS launch_count
FROM launches
GROUP BY name_x
ORDER BY launch_count DESC
LIMIT 5;
"""
result=pd.read_sql_query(q3, conn)
result

,name_x,launch_count
0,ispace Mission 1 & Rashid,1
1,ZUMA,1
2,WorldView Legion 1 & 2,1
3,Viasat-3 & Arcturus,1
4,USSF-44,1
